In [1]:
# Import library
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5,6,7"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import numpy as np
import pickle as pickle
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.recurrent import SimpleRNN, LSTM, GRU 
from keras import backend as K

with open('./train_data', 'rb') as f: 
    train_data = pickle.load(f) 
    train_label= pickle.load(f)
with open('./test_data', 'rb') as f: 
    test_data = pickle.load(f)

train_data.shape, test_data.shape

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2544778694765536128
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 169476096
locality {
  bus_id: 2
}
incarnation: 10438161854521502838
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:87:00.0"
, name: "/gpu:1"
device_type: "GPU"
memory_limit: 169476096
locality {
  bus_id: 2
}
incarnation: 14348223712715794654
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:8c:00.0"
, name: "/gpu:2"
device_type: "GPU"
memory_limit: 171573248
locality {
  bus_id: 2
}
incarnation: 5792057993565770289
physical_device_desc: "device: 2, name: Tesla K80, pci bus id: 0000:8d:00.0"
]


Using TensorFlow backend.


((50000, 3072), (10000, 3072))

In [ ]:
# # WRN

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5,6,7"


In [2]:
import numpy as np
import sklearn.metrics as metrics

import pickle as pickle
# import wide_residual_network as wrn
import wide_residual_network10 as wrn
from keras.datasets import cifar10
import keras.callbacks as callbacks
import keras.utils.np_utils as kutils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

from keras import backend as K

with open('./train_data', 'rb') as f: 
    train_data = pickle.load(f) 
    train_label= pickle.load(f)
with open('./test_data', 'rb') as f: 
    test_data = pickle.load(f)

train_data.shape, test_data.shape

((50000, 3072), (10000, 3072))

In [46]:
# trainX = training[:45000, 0:3072].reshape(45000, 32, 32, 3)
# trainX.shape
# trainY.shape
print(trainX.shape, a.shape)
print(trainY.shape, b.shape)
print(validY.shape, d.shape)
print(validX.shape, c.shape)

(45000, 32, 32, 3) (50000, 32, 32, 3)
(45000, 100) (50000, 10)
(5000, 100) (10000, 10)
(5000, 32, 32, 3) (10000, 32, 32, 3)


In [3]:
# X_train = np.reshape(train_data,(50000,32,32,3))
# print(X_train.shape)

y_train = np.ravel(train_label)
# print(len(y_train))
# len(np.unique(y_train))

training = np.append(train_data, y_train.reshape(50000,1), 1)

trainX = training[:45000, 0:3072].reshape(45000, 32, 32, 3)
trainY = training[:45000, -1]
validX = training[45000:, 0:3072].reshape(5000, 32, 32, 3)
validY = training[45000:, -1]

testX = np.reshape(test_data,(10000,32,32,3))
print(testX.shape, trainX.shape, validX.shape)

# normalization
trainX = trainX.astype('float32')
trainX = (trainX - trainX.mean(axis=0)) / (trainX.std(axis=0))
validX = validX.astype('float32')
validX = (validX - validX.mean(axis=0)) / (validX.std(axis=0))
print(testX.shape, trainX.shape, validX.shape)

trainY = kutils.to_categorical(trainY)
validY = kutils.to_categorical(validY)
print(trainY.shape, validY.shape)

(10000, 32, 32, 3) (45000, 32, 32, 3) (5000, 32, 32, 3)
(10000, 32, 32, 3) (45000, 32, 32, 3) (5000, 32, 32, 3)
(45000, 100) (5000, 100)


In [5]:
batch_size = 100
nb_epoch = 10
img_rows, img_cols = 32, 32


generator = ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

init_shape = (3, 32, 32) if K.image_dim_ordering() == 'th' else (32, 32, 3)

# For WRN-16-8 put N = 2, k = 8
# For WRN-28-10 put N = 4, k = 10
# For WRN-40-4 put N = 6, k = 4
model = wrn.create_wide_residual_network(init_shape, nb_classes=100, N=2, k=8, dropout=0.00)

model.summary()
# plot_model(model, "WRN-16-8.png", show_shapes=False)


Wide Residual Network-10-8 created.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 32, 16)    432         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 32, 32, 16)    64          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32, 32, 16)    0           batch_normalization_1[0][0]      
_______________________________________________________

In [6]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])
print("Finished compiling")

# model.load_weights("weights/WRN-16-8 Weights.h5")
# print("Model loaded.")

# model.fit_generator(generator.flow(trainX, trainY, batch_size=batch_size), steps_per_epoch=len(trainX) // batch_size, epochs=nb_epoch,
#                    callbacks=[callbacks.ModelCheckpoint("./WRN-16-8 Weights.h5",
#                                                         monitor="val_acc",
#                                                         save_best_only=True,
#                                                         verbose=1)],
#                    validation_data=(validX, validY),
#                    validation_steps=validX.shape[0] // batch_size,)

# model.fit_generator(generator.flow(trainX, trainY, batch_size=batch_size), steps_per_epoch=len(trainX) // batch_size, epochs=nb_epoch,
#                    callbacks=None,
#                    validation_data=(validX, validY),
#                    validation_steps=validX.shape[0] // batch_size,)
model.fit_generator(generator.flow(trainX, trainY, batch_size=batch_size), steps_per_epoch=len(trainX) // batch_size, epochs=nb_epoch,
                   callbacks=[callbacks.ModelCheckpoint("./WRN-16-8 Weights.h5",
                                                        monitor="val_acc",
                                                        save_best_only=True,
                                                        verbose=1)])

# yPreds = model.predict(validX)
# yPred = np.argmax(yPreds, axis=1)
# yPred = kutils.to_categorical(yPred)
# yTrue = validY

# accuracy = metrics.accuracy_score(yTrue, yPred) * 100
# error = 100 - accuracy
# print("Accuracy : ", accuracy)
# print("Error : ", error)

Finished compiling
Epoch 1/10
450/450 [==============================] - 361s - loss: 4.2132 - acc: 0.0521   

/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:405: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)



Epoch 2/10
450/450 [==============================] - 355s - loss: 3.6035 - acc: 0.1421   
Epoch 3/10
450/450 [==============================] - 355s - loss: 3.1900 - acc: 0.2134   
Epoch 4/10
450/450 [==============================] - 354s - loss: 2.9318 - acc: 0.2661   
Epoch 5/10
450/450 [==============================] - 352s - loss: 2.7169 - acc: 0.3043   
Epoch 6/10
450/450 [==============================] - 353s - loss: 2.5446 - acc: 0.3421   
Epoch 7/10
450/450 [==============================] - 352s - loss: 2.3994 - acc: 0.3730   
Epoch 8/10
450/450 [==============================] - 352s - loss: 2.2694 - acc: 0.3990   
Epoch 9/10
450/450 [==============================] - 351s - loss: 2.1450 - acc: 0.4258   
Epoch 10/10
450/450 [==============================] - 352s - loss: 2.0357 - acc: 0.4499   


In [7]:
prd_wrn = model.predict(testX)
prd_wrn_y = np.argmax(prd_wrn, axis=1)
print(prd_wrn_y[0:10])

[39 39 39 39 39 39 39 39 39 39]


In [8]:
np.savetxt("946_wrn.csv", prd_wrn_y, delimiter=",") # yield 0.35 on kaggle
model.save('./946_wrn.h5') # creates a HDF5 file 'CNN1.h5'

In [4]:
batch_size = 100
nb_epoch = 80
img_rows, img_cols = 32, 32


generator = ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

init_shape = (3, 32, 32) if K.image_dim_ordering() == 'th' else (32, 32, 3)

# For WRN-16-8 put N = 2, k = 8
# For WRN-28-10 put N = 4, k = 10
# For WRN-40-4 put N = 6, k = 4
model = wrn.create_wide_residual_network(init_shape, nb_classes=100, N=2, k=8, dropout=0.00)

model.summary()
# plot_model(model, "WRN-16-8.png", show_shapes=False)


Wide Residual Network-10-8 created.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 32, 16)    432         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 32, 32, 16)    64          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32, 32, 16)    0           batch_normalization_1[0][0]      
_______________________________________________________

In [5]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])
print("Finished compiling")

# model.load_weights("weights/WRN-16-8 Weights.h5")
# print("Model loaded.")

# model.fit_generator(generator.flow(trainX, trainY, batch_size=batch_size), steps_per_epoch=len(trainX) // batch_size, epochs=nb_epoch,
#                    callbacks=[callbacks.ModelCheckpoint("./WRN-16-8 Weights.h5",
#                                                         monitor="val_acc",
#                                                         save_best_only=True,
#                                                         verbose=1)],
#                    validation_data=(validX, validY),
#                    validation_steps=validX.shape[0] // batch_size,)

# model.fit_generator(generator.flow(trainX, trainY, batch_size=batch_size), steps_per_epoch=len(trainX) // batch_size, epochs=nb_epoch,
#                    callbacks=None,
#                    validation_data=(validX, validY),
#                    validation_steps=validX.shape[0] // batch_size,)
model.fit_generator(generator.flow(trainX, trainY, batch_size=batch_size), steps_per_epoch=len(trainX) // batch_size, epochs=nb_epoch,
                   callbacks=[callbacks.ModelCheckpoint("./WRN-16-8 Weights.h5",
                                                        monitor="val_acc",
                                                        save_best_only=True,
                                                        verbose=1)])


Finished compiling
Epoch 1/80
450/450 [==============================] - 373s - loss: 4.2314 - acc: 0.0511   

/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:405: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)



Epoch 2/80
450/450 [==============================] - 355s - loss: 3.6331 - acc: 0.1374   
Epoch 3/80
450/450 [==============================] - 354s - loss: 3.1967 - acc: 0.2152   
Epoch 4/80
450/450 [==============================] - 352s - loss: 2.9280 - acc: 0.2653   
Epoch 5/80
450/450 [==============================] - 352s - loss: 2.7300 - acc: 0.3049   
Epoch 6/80
450/450 [==============================] - 351s - loss: 2.5526 - acc: 0.3384   
Epoch 7/80
450/450 [==============================] - 352s - loss: 2.4089 - acc: 0.3696   
Epoch 8/80
450/450 [==============================] - 351s - loss: 2.2758 - acc: 0.3985   
Epoch 9/80
450/450 [==============================] - 350s - loss: 2.1543 - acc: 0.4204   
Epoch 10/80
450/450 [==============================] - 350s - loss: 2.0434 - acc: 0.4482   
Epoch 11/80
450/450 [==============================] - 349s - loss: 1.9419 - acc: 0.4723   
Epoch 12/80
450/450 [==============================] - 350s - loss: 1.8466 - acc: 0.494

In [21]:
testX = testX.astype('float32')
testX = (testX - testX.mean(axis=0)) / (testX.std(axis=0))

In [10]:
np.savetxt("946_wrn2.csv", prd_wrn1_y, delimiter=",") # yield 0.48 on kaggle
model.save('./946_wrn2.h5') # creates a HDF5 file '946_wrn2.h5'